In [7]:
import pandas as pd
import _pickle as cPickle

Importing data:
    Data Source="http://www.cs.cmu.edu/~dbamman/booksummaries.html"

Data set details:
    This dataset contains plot summaries for 16,559 books extracted
    from Wikipedia, along with aligned metadata from Freebase, including book author, title, and genre.

Column names:
    WikipediaArticleID 
    FreebaseID 
    BookTitle 
    Author 
    PublicationDate 
    BookGenres 
    Summary

In [8]:
data=pd.read_csv("C:\\Users\\sudhanshu\\Desktop\\Springboard\\Capstone 1\\BookReview\\booksummaries\\booksummaries.txt",sep="\t",header =None)

In [9]:
data.head()

,WikipediaArticleID,FreebaseID,BookTitle,Author,PublicationDate,BookGenres,Summary
0,620,/m/0hhy,Animal Farm,George Orwell,1945-08-17,"{""/m/016lj8"": ""Roman \u00e0 clef"", ""/m/06nbt"":...","Old Major, the old boar on the Manor Farm, ca..."
1,843,/m/0k36,A Clockwork Orange,Anthony Burgess,1962,"{""/m/06n90"": ""Science Fiction"", ""/m/0l67h"": ""N...","Alex, a teenager living in near-future Englan..."
2,986,/m/0ldx,The Plague,Albert Camus,1947,"{""/m/02m4t"": ""Existentialism"", ""/m/02xlf"": ""Fi...",The text of The Plague is divided into five p...
3,1756,/m/0sww,An Enquiry Concerning Human Understanding,David Hume,NaN,NaN,The argument of the Enquiry proceeds by a ser...
4,2080,/m/0wkt,A Fire Upon the Deep,Vernor Vinge,NaN,"{""/m/03lrw"": ""Hard science fiction"", ""/m/06n90...",The novel posits that space around the Milky ...


In [14]:
data.shape

(16559, 7)

Adding Column Names

In [8]:
data.columns = ["WikipediaArticleID", "FreebaseID", "BookTitle", "Author", "PublicationDate", "BookGenres", "Summary"]

In [11]:
data.isnull().sum()

WikipediaArticleID       0
FreebaseID               0
BookTitle                0
Author                2382
PublicationDate       5610
BookGenres            3718
Summary                  0
dtype: int64

Building LDA model to understand the kind of books in the data.

Model is build using Spacy and Gensim

Phrase modelling will be used for two levels, stopwords & punctuations will be removed and all words will be lemmatized

In [4]:
#Loading dependencies

import matplotlib.pyplot as plt
import gensim
import numpy as np
import spacy

from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
from gensim.models.wrappers import LdaMallet
from gensim.corpora import Dictionary
import pyLDAvis.gensim

import os, re, operator, warnings, codecs

from gensim.models import Phrases
from gensim.models.word2vec import LineSentence
from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore



from spacy.en import English
nlp = spacy.load("en")


In [16]:
#Forming sentence.txt file for book summaries

#Calculating Length of dataframe
DfLength=len(data)
#print(DfLength)
"""
Loop over every book summary and break it into sentences.
For each sentence, lematize the words, remove punctuations, remove numbers and save to 
a text file on the system
"""

directory = "C:\\Users\\sudhanshu\\Desktop\\Springboard\\Capstone 1\\BookReview\\booksummaries\\"
FileName="All_Sentences.txt"

filepath=directory+FileName



def punct_space(token):
    """
    helper function to eliminate tokens
    that are pure punctuation or whitespace
    """
    
    return token.is_punct or token.is_space


with codecs.open(filepath, 'w', encoding='utf_8') as f:
    #Iterating over each review
    for i in range(DfLength):
            #Providing each review to Spacy
            doc=nlp(data.at[i,"Summary"])
            #Breaking review into sentences and lemmatizing each word
            for sentences in doc.sents:
                #print(u" ".join([token.lemma_ for token in sentences if not punct_space(token)])) 
                string = u" ".join([token.lemma_ for token in sentences if not punct_space(token)])
                string = re.sub('[^a-zA-Z\n\.]', ' ', string)
                string = " ".join(string.split())
                f.write(string + "\n")
        

In [18]:
#Using Phrase modelling to have New York City appear as New_York_City (Two iterations of phrase modelling)

directory = "C:\\Users\\sudhanshu\\Desktop\\Springboard\\Capstone 1\\BookReview\\booksummaries\\"
FileName="All_Sentences.txt"

filepath=directory+FileName

##Using Gensim to read in the file using LineSentence

unigram_sentences = LineSentence(filepath)

#Forming a Bi-Gram model: New York becomes New_York

bigram_model = Phrases(unigram_sentences)

#Saving the Bi-Gram Model
bigram_model_filepath = os.path.join(directory, 'bigram_model_all')

bigram_model.save(bigram_model_filepath)

# load the finished Bi-Gram model from disk
bigram_model = Phrases.load(bigram_model_filepath)

#Saving the results of the Bi-Gram model on disk
bigram_sentences_filepath = os.path.join(directory, "bigram_sentences_all.txt")


#Writing Bi-Gram proccessed data to file
with codecs.open(bigram_sentences_filepath, 'w', encoding='utf_8') as f:
        
        for unigram_sentence in unigram_sentences:
            
            bigram_sentence = u' '.join(bigram_model[unigram_sentence])
            
            f.write(bigram_sentence + '\n')
            

#Using Gensim to read in the Bi-Gram sentence file using LineSentence

bigram_sentences = LineSentence(bigram_sentences_filepath)
          
#Forming a Tri-Gram model

Trigram_model = Phrases(bigram_sentences)

#Saving the Tri-Gram Model
Trigram_model_filepath = os.path.join(directory, 'Trigram_model_all')

Trigram_model.save(Trigram_model_filepath)

# load the finished Tri-Gram model from disk
Trigram_model = Phrases.load(Trigram_model_filepath)

#Saving the results of the Tri-Gram model on disk
Trigram_sentences_filepath = os.path.join(directory, "Trigram_sentences_all.txt")


#Writing Tri-Gram processed data to file and removing stopwords
   
with codecs.open(Trigram_sentences_filepath, 'w', encoding='utf_8') as f:
        
        for bigram_sentence in bigram_sentences:
            trigram_review = Trigram_model[bigram_sentence]
            trigram_review = [term for term in trigram_review if term not in spacy.en.STOPWORDS]
            Trigram_sentence = u' '.join(trigram_review)
            
            f.write(Trigram_sentence + '\n')

#Reading in the Tri-Gram sentences to learn its dictionary 

Trigram_dictionary_filepath = os.path.join(directory,"trigram_dict_all.dict")    

Trigram_reviews = LineSentence(Trigram_sentences_filepath)       
            
#Learning the dictionary by iterating over all reviews

Trigram_dictionary = Dictionary(Trigram_reviews)


#Removing tokens that are very rare or too common from
#The dictionary (filter_extremes) and reassign integer ids (compactify)

#Remove all words that are in less than 5 sentences or in are than 50% of the sentences 
Trigram_dictionary.filter_extremes(no_below=5, no_above=0.50)
Trigram_dictionary.compactify()

#Writing dictionary output to a file
Trigram_dictionary.save(Trigram_dictionary_filepath)

#Forming a Bag of Words implementation of the corpus
#Setting filepath to save the bag of words model
Trigram_bow_filepath = os.path.join(directory, 'trigram_bow_corpus_all.mm')

#Loading Dictionary from the disk
Trigram_dictionary = Dictionary.load(Trigram_dictionary_filepath)

#Forming a bag of words model
MmCorpus.serialize(Trigram_bow_filepath, (Trigram_dictionary.doc2bow(reviews) for reviews in LineSentence(Trigram_sentences_filepath)))          
          
#Loading the bag of words corpus from the disk
Trigram_bow_corpus = MmCorpus(Trigram_bow_filepath)  

#Forming LDA models

#Saving path name of the file
lda_model_filepath = os.path.join(directory, 'lda_model_all')


C:\Users\sudhanshu\Anaconda3\lib\site-packages\gensim\models\phrases.py:274: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


To better understand the typeof books in the data I have build a LDA model.
This model aims to find 10homogenious clusters in the data.

The LDA model was run for 6 to 20 clusters. But I have settled for 10 as beyond this number the 
clusters were breaking out into smaller sets that were hard to explain. 

In [21]:
#Starting LDA modelling
with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        
        # workers => sets the parallelism, and should be
        # set to your number of physical cores minus one
        #Looking for 10 topics
        Summarylda = LdaMulticore(Trigram_bow_corpus,
                           num_topics=10,
                           id2word=Trigram_dictionary,
                           workers=3)
#Saving the model to disk    
Summarylda.save(lda_model_filepath)

To Visualize the LDA model ldavis is used

In [22]:
# load the finished LDA model from disk
lda = LdaMulticore.load(lda_model_filepath)

#Preparing visualization for LDA

LDAvis_data_filepath = os.path.join(directory, 'ldavis_prepared')

LDAvis_prepared=pyLDAvis.gensim.prepare(lda, Trigram_bow_corpus, Trigram_dictionary)  

pyLDAvis.display(LDAvis_prepared)

To check if resonable structure is present in the text I have used Word2Vec model.
Forming Word2Vec model

In [24]:
from gensim.models import Word2Vec

trigram_sentences = LineSentence(Trigram_sentences_filepath)
word2vec_filepath = os.path.join(directory, 'word2vec_model_all')

In [26]:
# initiate the model and perform the first epoch of training

food2vec = Word2Vec(trigram_sentences, size=100, window=5, min_count=20, sg=1, workers=4)
    
food2vec.save(word2vec_filepath)

# perform another 12 epochs of training
for i in range(1,13):
    print("Running training epoch")
    print(i)
    food2vec.train(trigram_sentences)
    food2vec.save(word2vec_filepath)
        
# load the finished model from disk
food2vec = Word2Vec.load(word2vec_filepath)
food2vec.init_sims()


Running training epoch
1
Running training epoch
2
Running training epoch
3
Running training epoch
4
Running training epoch
5
Running training epoch
6
Running training epoch
7
Running training epoch
8
Running training epoch
9
Running training epoch
10
Running training epoch
11
Running training epoch
12


In [29]:
#Number of terms in vocab
food2vec.corpus_count

335152

Finding most similar words based on Word2Vec

In [31]:
def get_related_terms(token, topn=10):
    """
    look up the topn most similar terms to token
    and print them as a formatted list
    """

    for word, similarity in food2vec.most_similar(positive=[token], topn=topn):

        print(u'{:20} {}'.format(word, round(similarity, 3)))

In [182]:
get_related_terms(u'man')

woman                0.752
an_old               0.552
young_man            0.541
stranger             0.537
englishman           0.523
boy                  0.52
person               0.498
thug                 0.493
soldier              0.492
priest               0.483


Word Algebra

In [48]:
def word_algebra(add=[], subtract=[], topn=1):
    """
    combine the vectors associated with the words provided
    in add= and subtract=, look up the topn most similar
    terms to the combined vector, and print the result(s)
    """
    answers = food2vec.most_similar(positive=add, negative=subtract, topn=topn)
    
    for term, similarity in answers:
        print(term)

In [67]:
word_algebra(add=[u'husband', u'wife'], subtract=[u'happy'])

her_husband



Getting started with Doc2Vec

In [2]:
from gensim.models import doc2vec
from collections import namedtuple

C:\Users\sudhanshu\Anaconda3\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [259]:
docs = []

analyzedDocument = namedtuple('AnalyzedDocument', 'words tags')

In [260]:
for i in range(len(data)):
    document=nlp(data.at[i,"Summary"])
    tags=[i]
    unigram_review = [token.lemma_ for token in document if not punct_space(token)]
    bigram_review = bigram_model[unigram_review]
    trigram_review = Trigram_model[bigram_review]
    docs.append(analyzedDocument(trigram_review, tags))
    if i % 500 == 0:
        print ('Now summary is %s'%i)
#docs

C:\Users\sudhanshu\Anaconda3\lib\site-packages\gensim\models\phrases.py:274: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


Now summary is 0
Now summary is 500
Now summary is 1000
Now summary is 1500
Now summary is 2000
Now summary is 2500
Now summary is 3000
Now summary is 3500
Now summary is 4000
Now summary is 4500
Now summary is 5000
Now summary is 5500
Now summary is 6000
Now summary is 6500
Now summary is 7000
Now summary is 7500
Now summary is 8000
Now summary is 8500
Now summary is 9000
Now summary is 9500
Now summary is 10000
Now summary is 10500
Now summary is 11000
Now summary is 11500
Now summary is 12000
Now summary is 12500
Now summary is 13000
Now summary is 13500
Now summary is 14000
Now summary is 14500
Now summary is 15000
Now summary is 15500
Now summary is 16000
Now summary is 16500


In [261]:
# Training the model with size 100 vector and a context window of 50 words 
model = doc2vec.Doc2Vec(docs, size = 100, window = 50, min_count = 1, workers = 4)


In [266]:
#Checking word similarity
model.most_similar("father")

[('mother', 0.8109614849090576),
 ('grandfather', 0.7266961336135864),
 ('son', 0.6972399950027466),
 ('uncle', 0.6522113084793091),
 ('young_brother', 0.6432526111602783),
 ('brother', 0.6255658864974976),
 ('biological_father', 0.6185038089752197),
 ('old_brother', 0.6132087707519531),
 ('parent', 0.6006464958190918),
 ('granddaughter', 0.5586180686950684)]

In [267]:
# Training the model
# start training
for epoch in range(20):
    print ('Now training epoch %s'%epoch)
    model.train(docs)
    model.alpha -= 0.002  # decrease the learning rate
    model.min_alpha = model.alpha  # fix the learning rate, no decay
    if epoch % 5 == 0:
        print (epoch)


Now training epoch 0
0
Now training epoch 1
Now training epoch 2
Now training epoch 3
Now training epoch 4
Now training epoch 5
5
Now training epoch 6
Now training epoch 7
Now training epoch 8
Now training epoch 9
Now training epoch 10
10
Now training epoch 11
Now training epoch 12
Now training epoch 13
Now training epoch 14
Now training epoch 15
15
Now training epoch 16
Now training epoch 17
Now training epoch 18
Now training epoch 19


In [268]:
#Saving path name of the file
directory = "C:\\Users\\sudhanshu\\Desktop\\Springboard\\Capstone 1\\BookReview\\booksummaries\\TextFiles\\"
DocToVec_model_filepath = os.path.join(directory, 'DocToVec')
model.save(DocToVec_model_filepath)

In [5]:
#Uncomment the third line to run the code in continuation of the cell above. Comment last line for the same.
#Loading model
#model=gensim.models.doc2vec.Doc2Vec.load(DocToVec_model_filepath)

#Use the line below to load the saved model
model=gensim.models.doc2vec.Doc2Vec.load("C:\\Users\\sudhanshu\\Desktop\\Springboard\\Capstone 1\\BookReview\\booksummaries\\TextFiles\\DocToVec")

In [8]:
model.most_similar("father")

[('mother', 0.9025280475616455),
 ('son', 0.8302043080329895),
 ('brother', 0.8022996187210083),
 ('daughter', 0.778820276260376),
 ('family', 0.7785707712173462),
 ('he', 0.7773062586784363),
 ('in', 0.7636514902114868),
 ('be', 0.7592247128486633),
 ('have', 0.7495785355567932),
 ('and', 0.7495403289794922)]

Checking top 10 books similar to the book specified by index.
Mapping of Books and index is present in the Book Summary Data excel.

In [46]:
BookIndex=26
check=model.docvecs.most_similar(BookIndex,topn=10) 
print("Book Selected:")
print(data.loc[BookIndex,2])
print("  ")
print("Books Recommended:")
for i in range(10):
    print(data.loc[check[i][0],2])

Book Selected:
Dracula
  
Books Recommended:
House of Hell
Lord Kelvin's Machine
The Chemistry of Death
The Painted Man
The Soft Whisper of the Dead
Assassin
The Ringworld Throne
The Witch Hunters
Mr. Fairlie's Final Journey
Ravenloft


In [36]:
BookIndex=3736
check=model.docvecs.most_similar(BookIndex) 
print("Book Selected:")
print(data.loc[BookIndex,2])
print("  ")
print("Books Recommended:")
for i in range(10):
    print(data.loc[check[i][0],2])

Book Selected:
The Wars
  
Books Recommended:
Helmet for My Pillow
If I Die in a Combat Zone: Box Me Up and Ship Me Home
The Doctor In War
Insurrection
Zak's Lunch
Rage
Soldier's Heart
The Hallo-Wiener
Petey
The Settlers


In [23]:
BookIndex=158
check=model.docvecs.most_similar(BookIndex) 
print("Book Selected:")
print(data.loc[BookIndex,2])
print("  ")
print("Books Recommended:")
for i in range(10):
    print(data.loc[check[i][0],2])

Book Selected:
Harry Potter and the Philosopher's Stone
  
Books Recommended:
Fantastic Beasts and Where to Find Them
Harry Potter and the Prisoner of Azkaban
Harry Potter and the Half-Blood Prince
Harry Potter and the Goblet of Fire
GoodKnyght!
Talking to Dragons
Mistborn: The Alloy of Law
Dragons of Summer Flame
A Taste of Blackberries
Hop o' My Thumb


In [24]:
BookIndex=166
check=model.docvecs.most_similar(BookIndex) 
print("Book Selected:")
print(data.loc[BookIndex,2])
print("  ")
print("Books Recommended:")
for i in range(10):
    print(data.loc[check[i][0],2])

Book Selected:
Alice's Adventures in Wonderland
  
Books Recommended:
The Tale of Mrs. Tiggy-Winkle
Alice Through the Needle's Eye
Through the Looking-Glass
Alice in Verse: The Lost Rhymes of Wonderland
The Starlight Barking
Starting With Alice
The Matter of Araby in Medieval England
Everyone Poops
The Story of Miss Moppet
Damage


In [28]:
BookIndex=156
check=model.docvecs.most_similar(BookIndex) 
print("Book Selected:")
print(data.loc[BookIndex,2])
print("  ")
print("Books Recommended:")
for i in range(10):
    print(data.loc[check[i][0],2])

Book Selected:
Dr. No
  
Books Recommended:
The Man with the Golden Gun
Razor's Edge
Thunderball
A Concise Treatise on the Art of Angling
The Loveday Trials
Lucky Starr and the Oceans of Venus
The Matter of Araby in Medieval England
China Sky
Goldfinger
From Russia with Love


In [37]:
BookIndex=3953
check=model.docvecs.most_similar(BookIndex) 
print("Book Selected:")
print(data.loc[BookIndex,2])
print("  ")
print("Books Recommended:")
for i in range(10):
    print(data.loc[check[i][0],2])

Book Selected:
Star Wars Republic Commando: Hard Contact
  
Books Recommended:
The Clue of the Whistling Bagpipes
Babylon Revisited
Migraine
Insurrection
Tempest-Tost
Snap!
Irish Whiskey
Le lourd passé de Lagaffe
Guardians of Ga'Hoole Book 4: The Siege
Angel Light
